In [2]:
# Imports para manipulação de dados
import pandas as pd
import numpy as np
import joblib

# Import dos dados
import yfinance as yf

#Import para tratamento de datas
from datetime import datetime, timedelta

# Imports para visualização de dados
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib as m
import seaborn as sns

# Imports para análise e modelagem de séries temporais
from scipy import stats
from statsmodels.tsa.stattools import grangercausalitytests

# Filtra os warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Formatação para os gráficos e tabelas
plt.style.use('ggplot')
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
# Formatando os labels dos gráficos
m.rcParams['axes.labelsize'] = 14
m.rcParams['xtick.labelsize'] = 12
m.rcParams['ytick.labelsize'] = 12
m.rcParams['text.color'] = 'k'

In [5]:
data_inicio = datetime.now().replace(year=datetime.now().year - 10).strftime('%Y-%m-%d')
data_final = datetime.now().strftime('%Y-%m-%d')

In [6]:
goog = yf.download('GOOG',start='2015-09-21',end='2025-09-19',auto_adjust=False)
amzn = yf.download('AMZN',start='2015-09-21',end='2025-09-19',auto_adjust=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [7]:
goog.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2514 entries, 2015-09-21 to 2025-09-18
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (Adj Close, GOOG)  2514 non-null   float64
 1   (Close, GOOG)      2514 non-null   float64
 2   (High, GOOG)       2514 non-null   float64
 3   (Low, GOOG)        2514 non-null   float64
 4   (Open, GOOG)       2514 non-null   float64
 5   (Volume, GOOG)     2514 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 137.5 KB


In [8]:
goog.columns = ['Adj Close','close','High','Low','Open','Volume']
amzn.columns = ['Adj Close','close','High','Low','Open','Volume']

In [9]:
goog = goog[['close']]
amzn = amzn[['close']]
close = goog['close']

In [10]:
goog.describe().T

,count,mean,std,min,25%,50%,75%,max
close,2514.0,93.857684,49.040679,29.744499,52.589876,77.079498,135.10025,252.330002


In [11]:
datas = pd.date_range(start='2015-09-21', end='2025-09-19', freq='D')
dados = goog.copy()
dados= dados.reindex(datas)
dados.ffill(inplace=True)

### Teste Shapiro-Wilk

O teste Shapiro-Wilk é usado para verificar a normalidade dos dados. Ele testa a hipótese nula de que uma amostra veio de uma distribuição normal. O teste fornece um valor de estatística W e um valor-p.

W é a estatística de teste que mede a proximidade dos dados com a distribuição normal; valores próximos de 1 indicam que os dados são mais provavelmente normais.

Valor-p indica a probabilidade de obter um resultado extremo como o observado se a hipótese nula for verdadeira. Um valor-p baixo (geralmente menor que 0.05) leva à rejeição da hipótese nula, sugerindo que os dados não são provenientes de uma distribuição normal.

In [12]:
# Teste de Normalidade (Shapiro-Wilk)
shapiro_teste = stats.shapiro(close)
print(f"\nTeste Shapiro-Wilk: W={shapiro_teste[0]}, p-value={shapiro_teste[1]}")


Teste Shapiro-Wilk: W=0.9139606595545777, p-value=1.3782653994750667e-35


### Interpretação do Resultado:

W=0.9140376764426461: Indica um leve desvio da normalidade, já que está um pouco proximo de 1.

p-value=1.3990262095842183e-35: Um valor extremamente baixo, praticamente zero. Isso significa que há evidências muito fortes contra a hipótese nula e você pode concluir que os dados não seguem uma distribuição normal.

Portanto, você deve considerar que os dados analisados não são normalmente distribuídos e isso pode influenciar na escolha dos métodos estatísticos ou testes a serem aplicados posteriormente.

### Teste Anderson-Darling

O Teste Anderson-Darling é um teste estatístico utilizado para **verificar a normalidade de uma distribuição de dados**. Ele é frequentemente considerado mais poderoso que outros testes de normalidade, como o Shapiro-Wilk, principalmente em situações onde as amostras possuem caudas pesadas ou são assimétricas. A principal característica distintiva deste teste é que ele **foca mais nas caudas da distribuição** em comparação com outros métodos.

O teste fornece os seguintes componentes na sua saída:

*   **Estatística de Teste**: É um valor numérico que quantifica o grau de desvio dos dados observados em relação à distribuição normal esperada. Quanto maior este valor, maior o desvio da normalidade.
*   **Valores Críticos**: Apresenta uma lista de valores que são os limites de comparação para a estatística de teste. Esses valores críticos correspondem a diferentes níveis de significância.
*   **Níveis de Significância**: Uma lista de probabilidades (como 0.01, 0.05, 0.10, etc.) que estão associadas a cada um dos valores críticos. Esses níveis ajudam a determinar se o desvio observado é estatisticamente significativo.

Para interpretar o resultado: se a estatística de teste calculada for maior que o valor crítico para um determinado nível de significância (por exemplo, 0.05), isso sugere que se deve rejeitar a hipótese nula de que os dados seguem uma distribuição normal, indicando que os dados não são normalmente distribuídos a esse nível de confiança.

In [13]:
# Teste de Normalidade (Anderson-Darling)
ad_teste = stats.anderson(close)
print(f"\nTeste Anderson-Darling: Statistic={ad_teste.statistic}, Critical Values={ad_teste.critical_values}, Significance Level={ad_teste.significance_level}")


Teste Anderson-Darling: Statistic=76.35664986359825, Critical Values=[0.575 0.655 0.786 0.917 1.09 ], Significance Level=[15.  10.   5.   2.5  1. ]


### Interpretação do Resultado:

Estatística de teste = 76.18468176352462: Este valor é significativamente alto, indicando um forte desvio da normalidade.

Valores Críticos = [0.575 0.655 0.786 0.917 1.09 ]: Esses valores devem ser comparados à estatística de teste. Se a estatística de teste é maior que um valor crítico, isso indica rejeição da hipótese nula (que os dados são normais) nesse nível de significância.

Níveis de Significância = [15%, 10%, 5%, 2.5%, 1%]: Cada valor crítico corresponde a um desses níveis de significância. Quanto menor o nível de significância, mais forte a evidência necessária para rejeitar a hipótese nula.

Em nosso caso, a estatística de teste (76.18468176352462) é muito maior do que todos os valores críticos listados, até mesmo o mais alto associado a um nível de significância de 1%. Isso significa que a hipótese nula de normalidade pode ser rejeitada em todos esses níveis de significância.

Portanto, você pode concluir que os dados não são normalmente distribuídos. Esta informação é essencial para decidir quais técnicas estatísticas ou testes aplicar aos dados, já que muitos assumem a normalidade da distribuição dos dados.

### Teste de Levene

O teste de Levene é utilizado para verificar a homogeneidade das variâncias entre grupos. Ele é especialmente útil em análises que envolvem comparações entre grupos, como a ANOVA, onde uma das suposições é que todos os grupos devem ter variâncias iguais. O teste de Levene avalia a hipótese nula de que as variâncias dos grupos são iguais contra a hipótese alternativa de que pelo menos uma variância difere das outras.

Os resultados do teste incluem:

Estatística de teste: Um valor que quantifica o quanto as variâncias são diferentes entre os grupos. Valores maiores indicam diferenças maiores.

Valor-p: Indica a probabilidade de obter um resultado extremo como o observado se a hipótese nula for verdadeira. Um valor-p baixo sugere que pelo menos uma das variâncias é significativamente diferente.

In [14]:
split_index = len(close) // 2
levene_teste = stats.levene(close[:split_index], close[split_index:])
print(f"\nTeste de Levene: Statistic={levene_teste.statistic}, p-value={levene_teste.pvalue}")


Teste de Levene: Statistic=820.2730188905815, p-value=2.3347908667645175e-156


### Interpretação do Resultado:

**Estatística de teste = 815.3455742073438:** Este é um valor extremamente alto, indicando uma diferença substancial nas variâncias entre os grupos.

**Valor-p = 1.4742662981938904e-155:** Este valor é praticamente zero, o que significa que a hipótese nula de igualdade de variâncias é fortemente rejeitada.

Dada a estatística de teste muito alta e o valor-p extremamente baixo, você pode concluir com segurança que há uma diferença significativa nas variâncias entre os grupos avaliados. Este resultado sugere que as condições para técnicas estatísticas que requerem homogeneidade de variâncias, como ANOVA, não são satisfeitas. Portanto, pode ser necessário ajustar a análise ou aplicar métodos alternativos que não assumam homogeneidade de variâncias.

### Teste de Causalidade de Granger

O Teste de Causalidade de Granger é utilizado para determinar se uma série temporal é capaz de prever outra. Em nosso contexto, o teste foi aplicado para investigar se os preços de fechamento das ações da Apple (AAPL) podem prever os preços de fechamento das ações da Microsoft (MSFT).

Os resultados do teste são reportados para diferentes números de lags (atrasos) e cada teste avalia se os valores passados (lags) de uma série têm poder preditivo significativo sobre os valores futuros da outra série. Aqui estão os componentes chave do resultado para cada número de lags:

*   **Estatística F (ssr based F test):** Indica a significância estatística da previsão. Valores mais altos indicam uma relação mais forte.
*   **Valor-p:** Se o valor-p é baixo (tipicamente menor que 0.05), rejeita-se a hipótese nula de que os lags não ajudam a prever a série.
*   **Chi-quadrado (ssr based chi2 test):** Outra medida de significância que testa a mesma hipótese da estatística F, mas baseada em uma distribuição chi-quadrado.
*   **Teste da razão de verossimilhança:** Semelhante aos testes acima, mas usa uma abordagem logarítmica.

In [15]:
df_amzn = amzn.copy()
df_amzn= df_amzn.reindex(datas)
df_amzn.ffill(inplace=True)

In [16]:
df_uinidos = pd.concat([dados,df_amzn],axis=1)

In [17]:
df_uinidos.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3652 entries, 2015-09-21 to 2025-09-19
Freq: D
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   close   3652 non-null   float64
 1   close   3652 non-null   float64
dtypes: float64(2)
memory usage: 85.6 KB


In [18]:
df_uinidos.columns = ['goog_close', 'amzn_close']

In [19]:
# Aplicar o Teste de Causalidade de Granger
# O parâmetro 'maxlag' define o número máximo de lags a serem testados
maxlag = 5
test_result = grangercausalitytests(df_uinidos[['goog_close', 'amzn_close']], maxlag = maxlag)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.1038  , p=0.2935  , df_denom=3648, df_num=1
ssr based chi2 test:   chi2=1.1047  , p=0.2932  , df=1
likelihood ratio test: chi2=1.1045  , p=0.2933  , df=1
parameter F test:         F=1.1038  , p=0.2935  , df_denom=3648, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.9897  , p=0.3718  , df_denom=3645, df_num=2
ssr based chi2 test:   chi2=1.9822  , p=0.3712  , df=2
likelihood ratio test: chi2=1.9817  , p=0.3713  , df=2
parameter F test:         F=0.9897  , p=0.3718  , df_denom=3645, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.6497  , p=0.5831  , df_denom=3642, df_num=3
ssr based chi2 test:   chi2=1.9529  , p=0.5822  , df=3
likelihood ratio test: chi2=1.9524  , p=0.5824  , df=3
parameter F test:         F=0.6497  , p=0.5831  , df_denom=3642, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.6992  , p=0.

### Interpretação do Resultado:

Para cada número de lags, todas as métricas indicam que os preços das ações da Google(GOOG) não têm um poder preditivo significativo sobre os preços das ações da Amazon (AMZN). Isso é evidenciado pelos valores alto dos valores-p e baixas estatísticas de teste em todos os lags (de 1 a 5). Aqui estão algumas interpretações específicas para cada lag:

Lag 1: Os preços de fechamento da Amazon de um dia anterior não têm uma relação significativa com os preços de fechamento da oogle no dia seguinte. A estatística F é 0.9570 com um valor-p de 0.3237.

Lags 2-5: Ao aumentar o número de lags, a influência continua não sendo significativa, com a estatística F e os valores-p mantendo o mesmo comportamento em todos os níveis testados. A estatística F diminui e o valor-p permanece alto, sugerindo que a informação mais antiga da AMZN não possui relevância na previsão dos preços GOOG.